In [ ]:
import streamlit as st
import json
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os
from dotenv import load_dotenv

In [ ]:
def get_api_key():
    # .env 파일에서 환경 변수 로드
    load_dotenv()

    # 환경 변수 가져오기
    api_key = os.getenv("OPENAI_API_KEY")
    if api_key is None:
        print("🚨 OpenAI API 키가 설정되지 않았습니다! .env 파일을 확인하세요.")
    else:
        print("✅ OpenAI API 키가 정상적으로 로드되었습니다.")

get_api_key()

✅ OpenAI API 키가 정상적으로 로드되었습니다.


In [ ]:
def load_json_file():
    # JSON 데이터 로드 (과실비율 데이터)
    with open("accident_data.json", "r", encoding="utf-8") as file:
        accident_data = json.load(file)

    # JSON 데이터를 langchain Document로 변환
    documents = []
    for case in accident_data:
        doc = Document(
            page_content=f"사고유형: {case['사고유형']}\n자동차 A: {case['자동차 A']}\n자동차 B: {case['자동차 B']}\n사고 설명: {case['사고 설명']}\n과실 비율: {case['과실 비율']}"
        )
        documents.append(doc)

    # OpenAI Embeddings 사용하여 문서 벡터화
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorstore = FAISS.from_documents(documents, embeddings)

    return vectorstore


vectorstore = load_json_file()

for doc in vectorstore.similarity_search("신호위반", k=2): # k의 개수로 가져오는 문서 개수 조절 가능
    print(doc.page_content)
    selected_doc = doc.page_content

page_content='사고유형: 차2-2
자동차 A: 녹색 신호 직진
자동차 B: 녹색(적색)신호위반 좌회전
사고 설명: 신호기에 의해 교통정리가 이루어지고 있는 교차로에서 녹색신호에 직진하는 A차량과 맞 은편 방향에서 녹색신호에 좌회전(비보호 좌회전이 아님) 또는 적색신호에 좌회전하는 좌 회전 신호위반 B차량이 충돌한 사고이다.
과실 비율: A0 : B100'
page_content='사고유형: 차55-1
자동차 A: 녹색신호 직진
자동차 B: 적색신호 직진(긴급자동차)
사고 설명: 신호가 있는 교차로에서 A차량은 정상신호에 직진하고 긴급자동차인 B차량은 적색신호에 직진하다 발생하는 사고이다.
과실 비율: A60 : B40'


In [ ]:
# Streamlit UI
st.title("🚗 교통사고 과실비율 AI 챗봇")
st.write("사고 상황을 설명하면 AI가 과실비율을 알려드립니다.")

user_input = st.text_area("✏️ 사고 상황을 입력하세요", "")

In [12]:
# 검색기(Retriever) 생성
retriever = vectorstore.as_retriever()

# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다. (테스트용)
retriever.invoke("좌회전 차량과 직진차량과의 사고")

[Document(id='e473e8a1-60b6-4b67-8829-5ff099d2ad46', metadata={}, page_content='사고유형: 차13-2\n자동차 A: 직진(교차로 내 진로변경)\n자동차 B: 우회전\n사고 설명: 신호기에 의해 교통정리가 이루어지고 있지 않은 교차로에서 1차로를 따라 직진하다가 교차로 내에서 2차로로 진로변경을 하는 A차량과 A차량의 진행방향 우측도로에서 우회전을 하는 B차량이 충돌한 사고이다.\n과실 비율: A60 : B40'),
 Document(id='b7d43ea2-6997-45ca-a543-036377d46ec3', metadata={}, page_content='사고유형: 차21-1\n자동차 A: 좌회전(왼쪽차)\n자동차 B: 좌회전(오른쪽차)\n사고 설명: 양 차량이 교차로에서 동일방향으로 동시 또는 유사한 시각에 진행함에 있어, 크게 또는 작게 좌회전을 하다가 왼쪽에서 진행하는 A차량과 오른쪽에서 진 행하는 B차량이 충돌한 사고이다.\n과실 비율: A40 : B60'),
 Document(id='5607758f-89fe-4024-aee9-f5cbae33d460', metadata={}, page_content='사고유형: 차16-4\n자동차 A: 소로 직진(왼쪽 도로에서 진입)\n자동차 B: 대로 좌회전(오른쪽 도로에서 진입)\n사고 설명: 신호기에 의해 교통정리가 이루어지고 있지 않는 다른 폭의 교차로에서 소로를 이 용하여 직진하는 A차량과 A차량의 진행방향 오른쪽 대로를 이용하여 좌회전하는 B차량이 충돌한 사고이다.\n과실 비율: A50 : B50'),
 Document(id='d87d299b-390b-463e-afb3-56231ab40c9c', metadata={}, page_content='사고유형: 차17-1\n자동차 A: 좌회전(오른쪽 도로에서 진입)\n자동차 B: 좌회전(왼쪽 도로에서 진입)\n사고 설명: 신호기에 의해 교통정리가 이루어지고 있지 않는 동일 폭의 교차로에서 좌회전하는 A차량

In [ ]:
# PromptTemplate 설정
prompt = PromptTemplate.from_template("""
당신은 사고 상황을 설명하면 과실비율을 알려주는 챗봇입니다.
질문을 보고 참고 문서의 사고 설명과 가장 유사한 사고유형을 찾아 과실비율을 알려주세요.
유사한 사례가 없다면 "유사한 사례가 없습니다. 좀 더 구체적인 상황 설명이 필요합니다." 라고 말하세요.
대답은 한국어로 해주세요.

# 참고 문서: {document}
                                      
# 질문: {question}
"""
)

In [14]:
# 언어모델(LLM) 생성
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [15]:
# 체인(Chain) 생성
chain = (
    {"document": retriever, "question": user_input}
    | prompt
    | llm
    | StrOutputParser()     # 답을 항상 문자열로 출력
)

NameError: name 'user_input' is not defined

In [ ]:
if st.button("🚀 과실비율 분석하기"):
    result = chain.invoke(user_input)
    st.write(f"📌 AI 과실비율 결과: {result}")

In [17]:
!pip install nbconvert

In [ ]:
# ipynb를 py로 추가 저장
!jupyter nbconvert --to script run.ipynb -- output run.py

[NbConvertApp] WARNING | pattern 'output' matched no files
[NbConvertApp] WARNING | pattern 'run.py' matched no files
[NbConvertApp] Converting notebook run.ipynb to script
[NbConvertApp] Writing 2622 bytes to run.py
